In [1]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

# figure out the correct path
machop_path = Path(".").resolve().parent.parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

from chop.tools.checkpoint_load import load_model
from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph.analysis import (
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
)
from chop.passes.graph import (
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.ir.graph.mase_graph import MaseGraph

from chop.models import get_model_info, get_model

set_logging_verbosity("info")


/home/agomotto3000/anaconda3/envs/mase/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO     Set logging level to info


In [2]:
batch_size = 512
model_name = "toy_convnet"
dataset_name = "cifar10"

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
    # custom_dataset_cache_path="../../chop/dataset"
)
data_module.prepare_data()
data_module.setup()

model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False,
    checkpoint = None)

# LAB1_CUSTOM_PATH = "/home/bkt123/dev/advanced-deep-learning-systems/mase/mase_output/lab-1_jsc-custom/software/training_ckpts/best.ckpt"
# model = load_model(load_name=LAB1_CUSTOM_PATH, load_type="pl", model=model)

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
    max_batches=1
)

dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
# from chop.actions import train
# import torch

# # print(isinstance(mg.model, torch.nn.Module))

# model = mg.model
# model_info = get_model_info('jsc-tiny')
# dataset_info = get_dataset_info('jsc')
# task = "cls"

# train_params = {
#     "model": model,
#     "model_info": model_info,
#     "data_module": data_module,
#     "dataset_info": dataset_info,
#     "task": task,
#     "optimizer": "adam",
#     "learning_rate": 1e-3,
#     "weight_decay": 0,
#     "plt_trainer_args": {
#         "max_epochs": 1,
#     }, 
#     "auto_requeue": False,
#     "save_path": None,
#     "visualizer": None,
#     "load_name": None,
#     "load_type": None
# }

# train(**train_params)

In [4]:
from pprint import pprint

from chop.passes.graph.utils import get_mase_op

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

# pprint(mg.meta['mase'].__dict__)

for node in mg.fx_graph.nodes:
    if get_mase_op(node) == 'conv2d':
        print(node.name)
        print(50*'-')
        # print(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        # print(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        # print(node.meta['mase'].parameters['common']['args']['weight']['value'])
        # print(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])
        pprint(mg.modules[node.target].weight)
        pprint(mg.modules[node.target].state_dict())
        print(50*'-')

block_1_0
--------------------------------------------------
Parameter containing:
tensor([[[[-0.0281, -0.0582, -0.0371],
          [-0.1579,  0.0132, -0.0916],
          [-0.1409, -0.0279,  0.0260]],

         [[-0.0331,  0.1588,  0.0091],
          [ 0.1361, -0.0194,  0.0423],
          [-0.0636, -0.1731, -0.0179]],

         [[-0.1212,  0.1039, -0.1320],
          [-0.1762,  0.1007, -0.1141],
          [ 0.1080, -0.0230, -0.0278]]],


        [[[ 0.0323, -0.0230,  0.1359],
          [-0.0833,  0.1698, -0.0782],
          [-0.1471, -0.1869,  0.1094]],

         [[-0.0801,  0.0152,  0.0724],
          [ 0.0146,  0.0003, -0.0094],
          [-0.1833,  0.0050,  0.0855]],

         [[ 0.0945, -0.0190,  0.0050],
          [ 0.1021,  0.1551,  0.0910],
          [-0.1895, -0.1132, -0.1059]]],


        [[[-0.0246, -0.0291,  0.1384],
          [ 0.1425, -0.1888,  0.1047],
          [ 0.0226, -0.0433,  0.1525]],

         [[-0.1333,  0.0212,  0.1316],
          [-0.1676, -0.0624,  0.0050],
  

In [5]:
from chop.passes.graph.transforms import (
    prune_transform_pass,
)
pass_args = {
    "weight":{
        "scope" : "local",
        "granularity" : "channel",
        "method" :  "l1-norm",
        "sparsity" : 0.5,
    },
    "activation":{
        "scope" : "local",
        "granularity" : "elementwise",
        "method" : "l1-norm",
        "sparsity" : 0.5,
    },
}
 
mg, _ = prune_transform_pass(mg, pass_args)



tried
Reached3
Mask3
tensor([[[[False, False, False],
          [False, False, False],
          [False, False, False]],

         [[False, False, False],
          [False, False, False],
          [False, False, False]],

         [[False, False, False],
          [False, False, False],
          [False, False, False]]],


        [[[False, False, False],
          [False, False, False],
          [False, False, False]],

         [[False, False, False],
          [False, False, False],
          [False, False, False]],

         [[False, False, False],
          [False, False, False],
          [False, False, False]]],


        [[[False, False, False],
          [False, False, False],
          [False, False, False]],

         [[False, False, False],
          [False, False, False],
          [False, False, False]],

         [[False, False, False],
          [False, False, False],
          [False, False, False]]],


        [[[False, False, False],
          [False, False, False]

In [6]:
from pprint import pprint

from chop.passes.graph.utils import get_mase_op

# pprint(mg.meta['mase'].__dict__)

for node in mg.fx_graph.nodes:
    if get_mase_op(node) == 'conv1d':
        print(f"Layer: {node.name}")
        # pprint(node.meta['mase'].parameters['common'])
        # pprint(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        # pprint(node.meta['mase'].parameters['common']['args']['weight']['value'])
        pprint(mg.modules[node.target].weight)
        #pprint(mg.modules[node.target].parametrizations['weight'][0].mask)
        pprint(mg.modules[node.target].state_dict())
        # pprint(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])
        #print(mg.modules[node.target].parametrizations['weight'][0].mask == mg.modules[node.target].parametrizations['weight'][1].mask)
        total_w = 0
        pruned_w = 0
        w = mg.modules[node.target].weight
        for s in w:
            total_w += s.numel()
            pruned_w += s.numel() - s.nonzero().numel()

        pruned_percent = pruned_w / total_w
        print(f"Pruned percent: {pruned_percent}")

        print(50*'-')


Layer: block_3_0
Parameter containing:
tensor([[[-0.1409, -0.0761, -0.0573],
         [ 0.0117,  0.0768,  0.0178],
         [-0.1305,  0.0202, -0.0235],
         ...,
         [ 0.0556,  0.1262, -0.0789],
         [ 0.0327, -0.1376, -0.0521],
         [-0.0638,  0.0398, -0.0657]],

        [[-0.0781, -0.0895,  0.0452],
         [-0.1205,  0.0272, -0.0688],
         [-0.1330,  0.1047,  0.1162],
         ...,
         [-0.0790,  0.0293, -0.1086],
         [ 0.0287, -0.0769,  0.0658],
         [-0.1197,  0.0678, -0.0213]],

        [[ 0.0731, -0.1045, -0.0855],
         [ 0.0911, -0.0642,  0.0982],
         [-0.1269,  0.0221,  0.1263],
         ...,
         [-0.0627, -0.0403, -0.0903],
         [ 0.0504, -0.0632,  0.0472],
         [-0.1148, -0.0394,  0.0852]],

        ...,

        [[ 0.0984, -0.0615, -0.1379],
         [-0.0083,  0.0268, -0.1374],
         [ 0.0584,  0.1286,  0.0790],
         ...,
         [-0.0658,  0.1130,  0.0417],
         [-0.0559,  0.1345,  0.1233],
         [ 

In [7]:
# from pprint import pprint

# from chop.passes.graph.utils import get_mase_op

# # pprint(mg.meta['mase'].__dict__)

# for node in mg.fx_graph.nodes:
#     if get_mase_op(node) == 'linear':
#         print(f"Layer: {node.name}")
#         # pprint(node.meta['mase'].parameters['common'])
#         # pprint(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
#         # pprint(node.meta['mase'].parameters['common']['args']['weight']['value'])
#         pprint(mg.modules[node.target].weight)
#         # pprint(mg.modules[node.target].parametrizations['weight'][0].mask)
#         # pprint(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])

#         total_w = 0
#         pruned_w = 0
#         mask_2= mg.modules[node.target].parametrizations['weight'][0].mask
#         for s in mask_2:
#             total_w += s.numel()
#             pruned_w += s.numel() - s.nonzero().numel()

#         pruned_percent = pruned_w / total_w
#         print(f"Pruned percent: {pruned_percent}")

#         print(50*'-')


In [8]:
# from chop.actions import train
# import torch

# # print(isinstance(mg.model, torch.nn.Module))

# model = mg.model
# model_info = get_model_info('jsc-tiny')
# dataset_info = get_dataset_info('jsc')
# task = "cls"

# train_params = {
#     "model": model,
#     "model_info": model_info,
#     "data_module": data_module,
#     "dataset_info": dataset_info,
#     "task": task,
#     "optimizer": "adam",
#     "learning_rate": 1e-3,
#     "weight_decay": 0,
#     "plt_trainer_args": {
#         "max_epochs": 1,
#     }, 
#     "auto_requeue": False,
#     "save_path": None,
#     "visualizer": None,
#     "load_name": None,
#     "load_type": None
# }

# train(**train_params)

In [9]:
# from pprint import pprint

# from chop.passes.graph.utils import get_mase_op


# # pprint(mg.meta['mase'].__dict__)

# for node in mg.fx_graph.nodes:
#     if get_mase_op(node) == 'linear':
#         print(node.name)
#         print(50*'-')
#         # pprint(node.meta['mase'].parameters['common'])
#         # pprint(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
#         # pprint(node.meta['mase'].parameters['common']['args']['weight']['value'])
#         pprint(mg.modules[node.target].weight)
#         pprint(mg.modules[node.target].parametrizations['weight'][0].mask)
#         # pprint(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])

#         print(50*'-')